In [2]:
# Importing all needed libaries
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual,HBox, Label
import ipywidgets as widgets
from scipy.integrate import odeint as ode
import matplotlib.pyplot as plt
import numpy as np
import math

In [56]:
"""
Variables
"""
g = 32.174 #ft/s^2

W_max = 1030500 # lb
rho_sl = 0.002377693 #slug/ft^3
K = 0.055579171
C_L_max = 5 #2.7198
h_obstacle = 50 #ft
T_max = 388000 # take off thrust
mu = 0.05 #rolling resistance
C_D0 = 0.015 #historical
S_ref = 6891.60 #ft^2
C_L_min = 1.5
n=1.2 
step_size=0.01
"""
Functions
"""

def stall_vel(W, S,rho, C_L):
    return np.sqrt( (2*W_max) / (rho*S*C_L) )

def Lift(rho, V, C_L, S):
    return 1/2 * rho * C_L * S * V**2

def Drag(rho, V, S, C_D0, K, C_L):
    return 1/2*rho*V**2* (C_D0 + K*C_L**2)


# Equation 17.3 Raymer
def Endurance(L, D, C, W_i, W_f):
    return (L / D) * (1 / C) * np.log(W_i / W_f)

# Equation  17.23, Raymer
def Range(L, D, C, W_i, W_f, V):
    return Endurance(L,D,C,W_i,W_f) * V


### GROUND ROLL
def ground_accel(W, T, D, L, mu):
    return g / W * (T - D - mu*(W - L) )

def ground_dist(T, W, S, mu, rho, C_L, C_D0, K, V_I, V_F):
    
    K_T = (T / W) - mu
    K_A = (rho * ( mu*C_L - C_D0 - K*C_L**2 ) ) / ( 2 * ( W / S ) )
    
    return (1 / (2*g*K_A) )*np.log((K_T + K_A*V_F**2 ) / (K_T + K_A*V_I**2) ), K_T, K_A

# Transition
def Gamma(T, D, W):
    return math.asin( (T - D) / W )



### Steady, Level Flight. Cruise.

# Eq 17.10 Raymer
# Use this to determine optimal Cruise altitude
def cruise_vel(rho, C_L, W, S):
    return np.sqrt( (2 * W) / (rho*C_L * S))

def thrust_to_weight(q, C_D0, W, S, K):
    return q*C_D0 / (W / S) + (W / S) * (K / q)

def W_dot(C, T):
    return -C*T


## Fuel Required:
# C * T at the different stages of flight.
# make sure range requirement is met.


def Take_Off(T, W, S, mu, rho, C_L, C_D0, K, n=1.2):
    
    V_stall = stall_vel(W, S, rho, C_L)
    V_to = 1.1*V_stall
    V_trans = 1.15*V_stall
    D_to = 1/2*rho*V_to**2* (C_D0 + K*C_L**2)
    
    S_G, K_T, K_A = ground_dist(T, W, S, mu, rho, C_L, C_D0, K, 0, V_to)
    S_R = 3*V_to
    
    R_trans = V_trans**2 / (g * (n - 1))

    gamma_climb = np.arcsin( (T - D_to) / W )

    h_TR = R_trans*(1 - np.arccos(gamma_climb))

    if h_TR.any() > h_obstacle:
        S_TR = np.sqrt(R_trans**2 - (R_trans - h_obstacle)**2)
    else:
        S_TR = R_trans*np.arcsin(gamma_climb)


    takeoff_distance = S_G + S_R + S_TR
    
    return takeoff_distance, S_G, S_R, S_TR, h_TR, gamma_climb, V_stall, V_to, V_trans, K_T, K_A




def Take_Off_Interact(C_L_max, engine_num, W, S):
    
    T = T_max *engine_num/4

    C_L_array = np.arange(C_L_min, C_L_max+step_size, step_size)
    
    plt.figure(figsize=(8,7))
    plt.plot(C_L_array, Take_Off(T, W, S, mu, rho_sl, C_L_array, C_D0, K)[0], label='Total Distance')
    plt.plot(C_L_array, Take_Off(T, W, S, mu, rho_sl, C_L_array, C_D0, K)[1], label='Ground Roll', color='b')
    plt.plot(C_L_array, Take_Off(T, W, S, mu, rho_sl, C_L_array, C_D0, K)[3], label='Transition Distance', color='purple')
    plt.plot(C_L_array, Take_Off(T, W, S, mu, rho_sl, C_L_array, C_D0, K)[2], label='Rotation Ground Roll Distance',color='black')
    plt.xlabel('$C_L (-)$')
    plt.ylabel('Take-off Distance (ft)')
    plt.grid()
    plt.legend()
    #plt.plot(C_L_array, np.ones(len(C_L_array))*7000)
    plt.axhline(y=7000,color='r',linestyle='--')
    plt.annotate('7000 ft',[0.5, 7300])
    
    plt.figure(figsize=(8,7))
    plt.plot(C_L_array, Take_Off(T, W, S, mu, rho_sl, C_L_array, C_D0, K)[7], color='r')
    plt.xlabel('$C_L$ (-)')
    plt.ylabel('Take-Off Velocity (ft/sec)')
    plt.grid()

    return None

In [57]:
style = {'description_width': 'initial'}
interact_manual(Take_Off_Interact,
                C_L_max=widgets.FloatSlider(min=C_L_min+0.5, max=C_L_max,step=0.1,value=((C_L_min+C_L_max)/2),description='$C_{L_{max}}$',style=style),
                engine_num=widgets.IntSlider(min=3, max=4,step=1,value=3, description= 'Engine Number',style=style),
                W=widgets.FloatSlider(min=500000, max=1500000, step =100, value=W_max, description ='Take-Off Weight (lb)',style=style),
                S=widgets.FloatSlider(min=1000, max=15000, step =10, value=S_ref, description ='Wing Area ($ft^2$)',style=style),
               )

interactive(children=(FloatSlider(value=3.25, description='$C_{L_{max}}$', max=5.0, min=2.0, style=SliderStyle…

<function __main__.Take_Off_Interact(C_L_max, engine_num, W, S)>